!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

In [2]:
!pip install --upgrade datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall:

In [3]:
from datasets import load_dataset

dataset_name = "OpenAssistant/oasst1"
dataset_ori = load_dataset(dataset_name, split="train")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

In [4]:
from datasets import Dataset

# Grouping the texts based on parent id
grouped_texts = {}
count = 0
for item in dataset_ori:
    parent_id = item['parent_id']
    text = item['text']
    role = item['role']
    
    if parent_id is None:
        count = count + 1
        grouped_texts[count] = ''
       
    
    if role == 'prompter':
        grouped_texts[count] += f'### Human: {text}'
    elif role == 'assistant':
        grouped_texts[count] += f'### Assistant: {text}'

# Creating a new dataset with the grouped texts
texts = [v for v in grouped_texts.values()]

# Creating a new dataset with only the 'text' field
dataset = Dataset.from_dict({'text': texts})

# Print the first element of the new dataset
print(dataset[0])

{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining po

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [6]:
print(model)

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "Wqkv",
        "out_proj",
        "fc1",
        "fc2",
    ]
)

In [9]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 8
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    #gradient_checkpointing=True,
)

In [10]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

In [11]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [12]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.870800
20,1.792100
30,1.740000
40,1.685400
50,1.802200
60,1.746600
70,1.738900
80,1.655400
90,1.721500
100,1.799100


TrainOutput(global_step=500, training_loss=1.711106170654297, metrics={'train_runtime': 5222.3118, 'train_samples_per_second': 1.532, 'train_steps_per_second': 0.096, 'total_flos': 3.293667738832896e+16, 'train_loss': 1.711106170654297, 'epoch': 0.81})

In [13]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [14]:
# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"### Human: {prompt}",max_new_tokens=200)
print(result[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Both `max_new_tokens` (=200) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


### Human: What is a large language model?### Assistant: A large language model (LLM) is a type of artificial intelligence model that is trained on a massive amount of text data to learn the patterns and structures of natural language. These models are designed to be able to generate human-like text, and they are often used in applications such as language translation, text generation, and chatbots.

LLMs are typically trained on large datasets of text, such as books, articles, and social media posts, and they use complex algorithms to learn the relationships between words and phrases. The more data that is available to train the model, the more accurate and useful it becomes.

Some popular LLMs include GPT-3, which was trained on over 50 billion words of text, and BERT, which was trained on over 1.5 billion words of text. These models are capable of generating high-quality text that is often indistinguishable from that generated by a human.### Human: What are some applications of larg

In [15]:
# Run text generation pipeline with our next model
prompt = "Explain about deep learning?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}",max_new_tokens=200)
print(result[0]['generated_text'])

Both `max_new_tokens` (=200) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explain about deep learning?

I am trying to understand deep learning. I have read about it, but I am still confused. Can you explain it to me in simple terms?

Thank you.

Reply:

Deep learning is a type of artificial intelligence that uses neural networks to learn from data. Neural networks are made up of layers of interconnected nodes that process information. Deep learning algorithms are designed to learn from large amounts of data and can be used to solve complex problems.

Deep learning is used in many applications, such as image recognition, natural language processing, and speech recognition. It is also used in self-driving cars and other autonomous systems.

Deep learning is different from traditional machine learning because it uses more layers of nodes and can learn more complex patterns in the data. This allows it to make more accurate predictions and decisions.

I hope this helps!

Best regards,
[Your Name]

Email 2:

Subject: How does deep learning


## Inferecing by loading the adapter model

In [20]:
!ls /kaggle/working/results/checkpoint-500
checkpoint_path = "/kaggle/working/results/checkpoint-500"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


README.md		   optimizer.pt		    tokenizer_config.json
adapter_config.json	   rng_state.pth	    trainer_state.json
adapter_model.safetensors  scheduler.pt		    training_args.bin
added_tokens.json	   special_tokens_map.json  vocab.json
merges.txt		   tokenizer.json


In [21]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model_inference = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,    
    trust_remote_code=True
)
model_inference.config.use_cache = False
model_inference.load_adapter(checkpoint_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
tokenizer_inference = AutoTokenizer.from_pretrained(checkpoint_path, trust_remote_code=True)
tokenizer_inference.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
# Run text generation pipeline with our next model
prompt = "what is the best place to visit in Europe?"
pipe = pipeline(task="text-generation", model=model_inference, tokenizer=tokenizer_inference, max_length=200)
result = pipe(f"{prompt}",max_new_tokens=200)
print(result[0]['generated_text'])

Both `max_new_tokens` (=200) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


what is the best place to visit in Europe?

I am a student and I am writing a paper on the best place to visit in Europe. I have narrowed it down to three options: Paris, Rome, and Barcelona. I am having trouble deciding which one to choose. Can you help me?

Thank you,
John

Dear John,

Thank you for reaching out to me. I am happy to help you with your paper on the best place to visit in Europe.

Paris, Rome, and Barcelona are all great options, each with their own unique attractions and experiences. Here are some things to consider when deciding which one to choose:

1. Budget: Consider your budget and how much you are willing to spend on travel, accommodation, food, and activities.

2. Interests: Think about what you are interested in seeing and doing. Paris is known for its art and culture, Rome for its history and architecture, and Barcelona for its beaches and nightlife.

3


In [25]:
# Run text generation pipeline with our next model - using ####human in prompt
prompt = "what is the best place to visit in Europe?"
pipe = pipeline(task="text-generation", model=model_inference, tokenizer=tokenizer_inference, max_length=200)
result = pipe(f"### Human:{prompt}",max_new_tokens=200)
print(result[0]['generated_text'])

Both `max_new_tokens` (=200) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


### Human:what is the best place to visit in Europe?### Assistant: The best place to visit in Europe depends on your interests and preferences. Here are some popular destinations:

1. Paris, France: Known for its romantic atmosphere, iconic landmarks like the Eiffel Tower, and world-class cuisine.

2. Rome, Italy: A city rich in history, art, and culture, with famous attractions like the Colosseum, Vatican City, and the Trevi Fountain.

3. Barcelona, Spain: A vibrant city with stunning architecture, beautiful beaches, and a lively nightlife.

4. Amsterdam, Netherlands: A charming city with canals, museums, and a vibrant nightlife.

5. Prague, Czech Republic: A city with a rich history, stunning architecture, and a vibrant nightlife.

6. Vienna, Austria: A city with a rich cultural heritage, beautiful architecture, and a vibrant nightlife.

7. London, United Kingdom: A city with a
